In [5]:
import pypdf
import datetime

import pandas as pd

from pathlib import Path
from functools import partial
from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate
from utils_ccd import get_connection, get_info_file_path, extract_text_from_pdf

conn = get_connection()

In [18]:
llm = AzureChatOpenAI(model_name="gpt-4o")

def get_ementa(texto):
    try:
        prompt = PromptTemplate.from_template("""
        Você é um agente que analisa processos do Tribunal de Contas do Estado do Rio Grande do Norte e retira as ementas.
        Os processos analisados tem uma introdução em forma de ementa, com uma lista de temas.
        Retire a ementa do processo, que é o resumo do processo, e retorne apenas a ementa. Não adicione texto nenhum

        O texto da informação é o seguinte:
        "{input}"
        
        Sua resposta:
        """)

        chain = prompt | llm
        msg_obj = chain.invoke(texto)
        return msg_obj.content
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None

In [3]:
setor = 'CIP'
ano = 2025
meses = [4,5,6]

sql_informacoes_processos = f'''
SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
ppe.SequencialProcessoEvento as evento,
CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
inf.*
FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
    ON inf.idinformacao = ppe.idinformacao
WHERE setor = '{setor}'
and year(inf.data_resumo) = {ano}
and month(inf.data_resumo) IN ({', '.join(map(str, meses))})
'''
informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)
informacoes_processos['caminho_arquivo'] = informacoes_processos.apply(get_info_file_path, axis=1)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_17972\1340344764.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  informacoes_processos = pd.read_sql(sql_informacoes_processos, conn)


In [4]:
informacoes_setor = informacoes_processos[informacoes_processos['setor'].str.strip() == setor]
informacoes_setor['resumo'].fillna('', inplace=True)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_17972\1267597635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  informacoes_setor['resumo'].fillna('', inplace=True)


In [6]:
informacoes_setor = informacoes_setor[~informacoes_setor['resumo'].str.contains('Pagamento da parcela')]

informacoes_setor['texto'] = informacoes_setor['caminho_arquivo'].apply(extract_text_from_pdf)

processos_busca = ', '.join(f"'{proc}'" for proc in informacoes_setor['processo'].unique())

sql_all_informacoes_processos = f'''
SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
ppe.SequencialProcessoEvento as evento,
CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
inf.*
FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
    ON inf.idinformacao = ppe.idinformacao
WHERE concat(inf.numero_processo, '/', inf.ano_processo) in ({processos_busca})
'''
all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)
all_informacoes_processos['caminho_arquivo'] = all_informacoes_processos.apply(get_info_file_path, axis=1)

invalid pdf header: b'Geran'
incorrect startxref pointer(1)
parsing for Object Streams
C:\Users\05911205424\AppData\Local\Temp\ipykernel_17972\3642093708.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)


In [7]:
all_informacoes_processos

,arquivo,evento,processo,setor,numero_processo,ano_processo,ordem,data_resumo,resumo,sigilo,...,DataPublicacao,IdInformacaoSubstituida,UsuarioInformacaoSubstituida,DataInformacaoSubstituida,IdTipoParecer,Observacao,IdProcesso,E_Relatorio_Inicial,E_Relatorio_Auditoria,caminho_arquivo
0,DAM_FGO_701034_2012_0026.pdf,0,701034/2012,DAM_FGO,701034,2012,26,2016-05-12 07:59:19.557,Capa,None,...,2016-05-12 07:59:40.403,NaN,None,NaT,None,None,304646.0,None,None,\\10.24.0.6\tce$\Informacoes_PDF\DAM_FGO\DAM_F...
1,DAM_FGO_700627_2010_0016.pdf,1,700627/2010,DAM_FGO,700627,2010,16,2016-05-16 10:38:25.577,Processo Digitalizado (Resolução 01/2012)- Vol...,0,...,2016-05-16 10:38:41.847,NaN,None,NaT,None,None,275987.0,None,None,\\10.24.0.6\tce$\Informacoes_PDF\DAM_FGO\DAM_F...
2,DAM_FGO_701451_2011_0018.pdf,0,701451/2011,DAM_FGO,701451,2011,18,2016-05-23 10:24:12.887,Capa,None,...,2016-05-23 10:24:29.820,NaN,None,NaT,None,None,294402.0,None,None,\\10.24.0.6\tce$\Informacoes_PDF\DAM_FGO\DAM_F...
3,DAM_FGO_701123_2012_0024.pdf,1,701123/2012,DAM_FGO,701123,2012,24,2016-05-24 10:15:52.330,Processo Digitalizado (Resolução 01/2012)- Vol...,0,...,2016-05-24 10:16:10.767,NaN,None,NaT,None,None,304735.0,None,None,\\10.24.0.6\tce$\Informacoes_PDF\DAM_FGO\DAM_F...
4,DAM_FGO_700717_2011_0009.pdf,0,700717/2011,DAM_FGO,700717,2011,9,2016-08-03 10:51:07.727,Capa,None,...,2016-08-03 10:51:23.427,NaN,None,NaT,None,None,292445.0,None,None,\\10.24.0.6\tce$\Informacoes_PDF\DAM_FGO\DAM_F...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5610,GCPOT_005851_2010_0053.pdf,35,005851/2010,GCPOT,005851,2010,53,2025-06-16 11:40:57.310,Despacho,N,...,2025-06-16 13:22:49.480,NaN,None,NaT,None,None,NaN,None,None,\\10.24.0.6\tce$\Informacoes_PDF\GCPOT\GCPOT_0...
5611,DAE_MANDA_001879_2024_0020.pdf,25,001879/2024,DAE_MANDA,001879,2024,20,2024-08-30 12:56:41.153,Citação nº 001471/2024 - DAE - DACIO TAVARES D...,0,...,2024-08-30 12:56:59.520,NaN,None,NaT,None,None,NaN,None,None,\\10.24.0.6\tce$\Informacoes_PDF\DAE_MANDA\DAE...
5612,GCGIL_008145_2003_0049.pdf,44,008145/2003,GCGIL,008145,2003,49,2025-04-10 10:11:37.743,...,None,...,2025-04-11 12:06:41.733,NaN,None,NaT,None,None,NaN,None,None,\\10.24.0.6\tce$\Informacoes_PDF\GCGIL\GCGIL_0...
5613,PROC_CTJ_011811_2002_0064.pdf,54,011811/2002,PROC_CTJ,011811,2002,64,2025-06-04 10:27:17.710,EMENTA.....,None,...,2025-06-10 11:22:01.640,NaN,None,NaT,None,None,NaN,None,None,\\10.24.0.6\tce$\Informacoes_PDF\PROC_CTJ\PROC...


In [8]:
def get_next_informacao(processo, df_last, df_informacoes):
    try:
        evento = df_last[df_last['processo'] == \
                                         processo]['evento'].iloc[0]
        return df_informacoes[(df_informacoes.evento > \
                                      evento) & \
                                        (df_informacoes['processo'] == \
                                         processo)].sort_values(by='evento').iloc[0]
    except IndexError:
        return None

def get_text_next_informacao(processo, df_last, df_informacoes):
    info = get_next_informacao(processo, df_last, df_informacoes)
    if info is not None:
        return extract_text_from_pdf(info['caminho_arquivo'])
    return ''

get_text_next_informacao_partial = partial(get_text_next_informacao, \
                                               df_last=informacoes_setor, \
                                                df_informacoes=all_informacoes_processos)
informacoes_setor['proximo_texto'] = informacoes_setor['processo'].apply(get_text_next_informacao_partial)


invalid pdf header: b'Geran'
incorrect startxref pointer(1)
parsing for Object Streams
invalid pdf header: b'Geran'
incorrect startxref pointer(1)
parsing for Object Streams
invalid pdf header: b'Geran'
incorrect startxref pointer(1)
parsing for Object Streams


In [25]:
informacoes_setor['ementa'] = informacoes_setor['texto'].apply(lambda x: get_ementa(x))

In [26]:
informacoes_setor.columns

Index(['arquivo', 'evento', 'processo', 'setor', 'numero_processo',
       'ano_processo', 'ordem', 'data_resumo', 'resumo', 'sigilo', 'usuario',
       'data_ultima_atualizacao', 'informacao_efetuada_por', 'nome_informacao',
       'Titulo_Modelo_informacao', 'Tipo_documento', 'Decisao', 'idInformacao',
       'infConvPdf', 'dataConversaoPDF', 'codigo_camara', 'IdSessao',
       'DataInclusao', 'UsuarioInclusao', 'ErroConversao', 'Assinado',
       'ProcessoEletronico', 'Publicado', 'IdModelo', 'Inativa',
       'DataPublicacao', 'IdInformacaoSubstituida',
       'UsuarioInformacaoSubstituida', 'DataInformacaoSubstituida',
       'IdTipoParecer', 'Observacao', 'IdProcesso', 'E_Relatorio_Inicial',
       'E_Relatorio_Auditoria', 'caminho_arquivo', 'texto', 'proximo_texto',
       'ementa'],
      dtype='object')

In [32]:
processos_busca = ', '.join(f"'{proc}'" for proc in informacoes_setor['processo'].unique())
sql_processos_mais = f'''
SELECT concat(numero_processo, '/', ano_processo) as processo, 
r.nome as relator, 
p.codigo_tipo_processo,
o.nome as orgao_envolvido
FROM processo.dbo.Processos p
INNER JOIN processo.dbo.Relator r ON p.codigo_relator = r.codigo
INNER JOIN processo.dbo.Orgaos o ON o.IdOrgao = p.IdOrgaoEnvolvido 
WHERE concat(numero_processo, '/', ano_processo) in ({processos_busca})

'''
df_processos_mais = pd.read_sql(sql_processos_mais, conn)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_17972\1438606903.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_processos_mais = pd.read_sql(sql_processos_mais, conn)


In [34]:
tab_cip = informacoes_setor.merge(df_processos_mais, on='processo', how='left')

In [36]:
tab_cip[['processo', 'codigo_tipo_processo', 'relator', 'orgao_envolvido', 'ementa']].to_excel('saidas/ementas_cip.xlsx', index=False)